In [1]:
# Feb 2017 - Ben Grauer 
# Created a python script that does some basic exploratory analysis / dov and exports to excel for easy viewing
# Originally designed for the lending club data set from Kaggle - Still a work in progress

In [ ]:
# Next Steps
# - for the correlation.  Put anything above .50 in an orange / light blue status.  Then anything more in deep red/blue
# - Put a notes section for whether to keep or not. 
# - Include a possibility for a list of columns to go in to keep.
# - Identify the continous, vs descrete
# - If you have classifiers, then show distribution against that (if a binary classification)
# - Add conditional formatting to correlation to see highly correlated or negative correlated
# - add something that will indicate if there are major groupings (by percentage) - mainly want to see if groups are differnt
# - reg expression items
# -- something that detects years (1900 - 2000), the word year or word month (with 75% success)

# INTELLIGENT NOTES
# -- If 95% is null, cut them out

# BUGS
# - for n/a it is calculating percentage incorrectly - see mths_since_last_delinq

In [2]:
import xlsxwriter
import pandas as pd
from xlsxwriter.utility import xl_rowcol_to_cell

In [3]:
dfLoan = pd.read_csv('/project/data/kg_lendingclub/loan.csv')
fileName = '/project/data/kg_lendingclub/LendingClub_DOV.xlsx'
#dfLoan = pd.read_csv('/Project/data/kg_titantic/train.csv')
#fileName = '/Project/data/kg_titantic/DOV_Titantic.xlsx'

//anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Function to take a workbook, sheetname, and iterate through to populate columns
# TODO: Break out by number:  array([dtype('int64'), dtype('float64'), dtype('O')], dtype=object)
#      dfLoan.dtypes
def excel_add_df(inputWorkBook, inputWorkSheet, inputSheetName, inputDataFrame, 
                 inputStartRow=0,inputStartCol=0, inputUseIndex=False):
    
    #bold = inputWorkBook.add_format({'bold': True})
    font14bold = inputWorkBook.add_format({'font_size':14, 'bold': True})
    
    # local variables
    rowNum = inputStartRow
    colNum = inputStartCol
    
    # add the input header about the sheet
    inputWorkSheet.write(rowNum, inputStartCol, inputSheetName, font14bold)
    rowNum = rowNum + 1
    
    # add an index
    if inputUseIndex==True:
        inputWorkSheet.write_column(inputStartRow+3, colNum, inputDataFrame.index, bold)
        colNum = colNum + 1
    
    for column in inputDataFrame:
        rowNum = inputStartRow + 2  # for the column headers. so 2 down
        inputWorkSheet.write(rowNum, colNum, inputDataFrame[column].name, bold)
        
        rowNum = inputStartRow + 3
        inputWorkSheet.write_column(rowNum, colNum, inputDataFrame[column])
    
        # add a column
        colNum = colNum + 1

In [5]:
# Things to do as time permits
# - add a box around everything
# - add numbers to each of the percentages
# - 

# open the excel sheet
#workbook = xlsxwriter.Workbook('LendingClub_DOV.xlsx', {'nan_inf_to_errors': True})
workbook = xlsxwriter.Workbook(fileName, {'nan_inf_to_errors': True})

worksheet = workbook.add_worksheet('DOV')


# Add a bold format to use to highlight cells.
bold = workbook.add_format({'bold': True})
italic = workbook.add_format({'italic': True})
underline = workbook.add_format({'underline': True})
formatHigh = workbook.add_format({'font_color': 'red'})
#formatHigh.set_pattern(1)
#formatHigh.set_fg_color('rose')
formatLow = workbook.add_format({'font_color': 'blue'})
#formatLow.set_pattern(1)
#formatLow.set_fg_color('ice blue')

# star the column Iteration at 2
colIteration = 1
rowIteration = 0 # to start
rowDataHeader = 6 # bumpt out to 5 to start, so we have 4 for summary data

# set the row descriptions
worksheet.write(rowIteration, 0, 'Col Name')
worksheet.write(rowIteration+1, 0, 'Data Type')
worksheet.write(rowIteration+2, 0, '# Na')
worksheet.write(rowIteration+3, 0, 'Cont_Disc') # left to fill out?  may replce with something else


for column in dfLoan:
    worksheet.write(rowIteration, colIteration, dfLoan[column].name, bold) # header
    worksheet.write(rowIteration+1, colIteration, str(dfLoan[column].dtypes)) # data type
    
    # TODO: # of na's 
    count_nan = len(dfLoan[column]) - dfLoan[column].count()
    if count_nan > 0:
        count_nan = str(round(( (len(dfLoan) - dfLoan[column].count()) / len(dfLoan[column]))*100, 0)) + '% - ' + str(count_nan)  
        
        
    worksheet.write(rowIteration+2, colIteration, count_nan) # data type
    # TODO: continous or discrete
    # number of unique values
    # count the number of nan
    # add the describe function for those that work
    # dfLoan.describe()b
    
    # want to detect categoirical or continuous.
    # if < 25 % is unique then categorical
    # if > 75 % is not unique then continouse, also check numbers
    
    # header for the DOV
    worksheet.write(rowDataHeader-1, colIteration, 'DOV', underline)
    
    
    if dfLoan[column].nunique() > 500:
        worksheet.write(rowDataHeader, colIteration, '> 500 unq', italic)
        worksheet.write_column(rowDataHeader+1, colIteration, dfLoan[column].head(500))
        colIteration = colIteration + 1
        
        # here need to determine %
        
    else:

        # grab the distribution percentage
        disbDF = pd.DataFrame(dfLoan.groupby([column]).size() * 100 / len(dfLoan))
        disbDF.rename(columns={0:'distprc'}, inplace=True)
        disbDF = disbDF.sort_values(['distprc'], ascending=False)
        
        # write the distribution header + percentage
        
        # TODO: change the index name to "DOV" for better readability
        worksheet.write_column(rowDataHeader, colIteration, disbDF.index) # here we are adding the index which is the DOV
        colIteration = colIteration + 1
        
        worksheet.write(rowDataHeader-1, colIteration, 'DistPrc')
        worksheet.write_column(rowDataHeader, colIteration, disbDF.loc[: ,'distprc'])
        colIteration = colIteration + 1

        
# Summary Stats
worksheet = workbook.add_worksheet('SummaryStats')
excel_add_df(inputWorkBook=workbook, inputWorkSheet=worksheet, inputSheetName='SummaryStats', 
             inputDataFrame=dfLoan.describe(),inputStartRow=0,inputStartCol=0,inputUseIndex=True)
        
# Write out the correlation matrix work-sheet
worksheet = workbook.add_worksheet('Correlation')
dfCorr = dfLoan.corr()
excel_add_df(inputWorkBook=workbook, inputWorkSheet=worksheet, inputSheetName='Correlation', 
             inputDataFrame=dfCorr,inputStartRow=0,inputStartCol=0,inputUseIndex=True)

# add conditional highlighting
# length of columns
lenColsDfCorr = len(dfCorr.columns) + 1
lenRowsDfCorr = len(dfCorr.index) + 4 # for header
cellStart = xl_rowcol_to_cell(3, 2)  # C2
cellEnd = xl_rowcol_to_cell(len(dfCorr.index) + 2, len(dfCorr.columns) )  # C2

worksheet.conditional_format(cellStart + ':' + cellEnd, {'type':     'cell',
                                        'criteria': '>=',
                                        'value':    0.75,
                                        'format':   formatHigh})

worksheet.conditional_format('B3:K12', {'type':     'cell',
                                        'criteria': '<',
                                        'value':    -0.75,
                                        'format':   formatLow})


# Write out the co-variance matrix work-sheet
worksheet = workbook.add_worksheet('Co-Variance')
excel_add_df(inputWorkBook=workbook, inputWorkSheet=worksheet, inputSheetName='Co-Variance', 
             inputDataFrame=dfLoan.cov(),inputStartRow=0,inputStartCol=0,inputUseIndex=True)


# Write out the co-variance matrix work-sheet
worksheet = workbook.add_worksheet('100samples')
excel_add_df(inputWorkBook=workbook, inputWorkSheet=worksheet, inputSheetName='100samples', 
             inputDataFrame=dfLoan.head(100),inputStartRow=0,inputStartCol=0,inputUseIndex=True)



#writer.save()
workbook.close()

print('Done!')

C4
AZ54
Done!
